In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.neural_network import MLPClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna("S")

In [6]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
train1 = train.drop(['PassengerId','Survived','Name','Ticket','Cabin'], axis=1)
col1 = [n for n in train1.columns if 
                                train1[n].nunique() < 10 and
                                train1[n].dtype == "object"]
col2 = [n for n in train1.columns if 
                                train1[n].dtype in ['int64', 'float64']]
cols = col1 + col2
train_pre = train1[cols]

dummy = pd.get_dummies(train1)

In [8]:
y_target = train["Survived"].values
x_features_one = dummy.values
FM = []

In [9]:
def treeDecision():
    print("Tree Decision")
    x_train, x_validation, y_train, y_validation = train_test_split(x_features_one,y_target,test_size=.20,random_state=1)
    tree1 = tree.DecisionTreeClassifier()
    tree1 = tree1.fit(x_features_one, y_target)
    predictions = tree1.predict(x_validation)
    recall = recall_score(y_validation,predictions)
    precision = precision_score(y_validation,predictions)
    f_measure = f1_score(y_validation,predictions)
    print("Recall",recall)
    print("Precision",precision)
    print("F Measure",f_measure)
    FM.append(f_measure)

In [10]:
def naive():
    print("Naive")
    x_train, x_validation, y_train, y_validation = train_test_split(x_features_one,y_target,test_size=.20,random_state=1)
    gnb = GaussianNB()
    gnb = gnb.fit(x_features_one, y_target)
    predictions = gnb.predict(x_validation)
    recall = recall_score(y_validation,predictions)
    precision = precision_score(y_validation,predictions)
    f_measure = f1_score(y_validation,predictions)
    print("Recall",recall)
    print("Precision",precision)
    print("F Measure",f_measure)
    FM.append(f_measure)

In [11]:
def neural():
    print("Neural")
    x_train, x_validation, y_train, y_validation = train_test_split(x_features_one,y_target,test_size=.20,random_state=1)
    clf = MLPClassifier()
    clf = clf.fit(x_features_one, y_target)
    predictions = clf.predict(x_validation)
    recall = recall_score(y_validation,predictions)
    precision = precision_score(y_validation,predictions)
    f_measure = f1_score(y_validation,predictions)
    print("Recall",recall)
    print("Precision",precision)
    print("F Measure",f_measure)
    FM.append(f_measure)

In [12]:
treeDecision()
naive()
neural()
print('AVERAGE F Measure ',sum(FM)/len(FM))

Tree Decision
Recall 0.9041095890410958
Precision 1.0
F Measure 0.949640287769784
Naive
Recall 0.684931506849315
Precision 0.7246376811594203
F Measure 0.704225352112676
Neural
Recall 0.7397260273972602
Precision 0.782608695652174
F Measure 0.7605633802816901
AVERAGE F Measure  0.80480967338805


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
